In [1]:
import sys #access to system parameters https://docs.python.org/3/library/sys.html
print("Python version: {}". format(sys.version))

import pandas as pd #collection of functions for data processing and analysis modeled after R dataframes with SQL like features
print("pandas version: {}". format(pd.__version__))

import matplotlib #collection of functions for scientific and publication-ready visualization
import matplotlib as plt
print("matplotlib version: {}". format(matplotlib.__version__))

import numpy as np #foundational package for scientific computing
print("NumPy version: {}". format(np.__version__))

import scipy as sp #collection of functions for scientific computing and advance mathematics
print("SciPy version: {}". format(sp.__version__)) 

import IPython
from IPython import display #pretty printing of dataframes in Jupyter notebook
print("IPython version: {}". format(IPython.__version__)) 

import sklearn #collection of machine learning algorithms
print("scikit-learn version: {}". format(sklearn.__version__))

#misc libraries
import random
import time
#ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)
import re
import os
import matplotlib.pyplot as plt
%matplotlib inline

Python version: 3.9.7 (v3.9.7:1016ef3790, Aug 30 2021, 16:39:15) 
[Clang 6.0 (clang-600.0.57)]
pandas version: 1.3.4
matplotlib version: 3.4.3
NumPy version: 1.21.3
SciPy version: 1.7.1
IPython version: 7.29.0
scikit-learn version: 1.0.1
-------------------------


In [2]:
#url = 'https://data.cityofnewyork.us/api/views/qgea-i56i/rows.csv?accessType=DOWNLOAD'
fn_src = 'https://data.cityofnewyork.us/api/views/uip8-fykc/rows.csv?accessType=DOWNLOAD'
fn_dst = 'NYPD_Arrest_Data.csv'

from six.moves import urllib

if os.path.isfile(fn_dst):
    print('File %s has already been downloaded' % fn_dst)
else:
    print('Fetching file %s[2.4GB]. This may take a while...' % fn_dst)
    urllib.request.urlretrieve(fn_src, fn_dst)
    print('File %s has been downloaded' % fn_dst)

Fetching file NYPD_Arrest_Data.csv[2.4GB]. This may take a while...
File NYPD_Arrest_Data.csv has been downloaded


### Open a stream

In [65]:
from openclean.pipeline import stream
ds_full = stream(fn_dst)

In [66]:
print(f'The dataset contains {ds_full.count():,} rows.')

The dataset contains 115,299 rows.


In [67]:
ds_full.head()

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,New Georeferenced Column
0,234233843,09/29/2021,105,STRANGULATION 1ST,106,FELONY ASSAULT,PL 1211200,F,B,42,0,25-44,M,BLACK,1009231,240290,40.82618898100003,-73.90973778899996,POINT (-73.90973778899996 40.82618898100003)
1,234129823,09/27/2021,157,RAPE 1,104,RAPE,PL 1303501,F,K,77,0,25-44,M,BLACK,1003606,185050,40.67458330800008,-73.93022154099998,POINT (-73.93022154099998 40.67458330800008)
2,234040747,09/25/2021,109,"ASSAULT 2,1,UNCLASSIFIED",106,FELONY ASSAULT,PL 1200501,F,Q,101,0,25-44,M,BLACK,1049232,159210,40.60344094100003,-73.76598558899997,POINT (-73.76598558899997 40.60344094100003)
3,234047720,09/25/2021,101,ASSAULT 3,344,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,B,44,0,25-44,M,BLACK,1006537,244511,40.83778161800007,-73.91945797099999,POINT (-73.91945797099999 40.83778161800007)
4,234042526,09/25/2021,101,ASSAULT 3,344,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,B,44,0,25-44,M,BLACK,1007418,243859,40.83598980000005,-73.91627635999998,POINT (-73.91627635999998 40.83598980000005)
5,234006839,09/24/2021,582,,,,PL 2225501,F,M,23,0,25-44,M,BLACK,998659,228177,40.79296572700008,-73.94796350499998,POINT (-73.94796350499998 40.79296572700008)
6,233952333,09/23/2021,397,"ROBBERY,OPEN AREA UNCLASSIFIED",105,ROBBERY,PL 1600500,F,K,75,0,18-24,M,BLACK,1017119,183909,40.67141166300007,-73.88151172399995,POINT (-73.88151172399995 40.67141166300007)
7,233878488,09/21/2021,105,STRANGULATION 1ST,106,FELONY ASSAULT,PL 1211300,F,B,40,0,25-44,M,WHITE,1010679,234533,40.81038342800008,-73.90452841699994,POINT (-73.90452841699994 40.81038342800008)
8,233784866,09/19/2021,101,ASSAULT 3,344,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,B,44,0,25-44,F,BLACK,1004489,237844,40.81948754300004,-73.926879392,POINT (-73.926879392 40.81948754300004)
9,233647454,09/16/2021,397,"ROBBERY,OPEN AREA UNCLASSIFIED",105,ROBBERY,PL 1601503,F,K,61,0,18-24,M,BLACK,995834,161062,40.60875497300003,-73.95828138999997,POINT (-73.95828138999997 40.60875497300003)


#### Use openclean to profile the data set:


In [68]:
from openclean.profiling.column import DefaultColumnProfiler

#profiles = ds.profile(default_profiler=DefaultColumnProfiler)
profiles = ds_full.profile(default_profiler=DefaultColumnProfiler)

In [69]:
profiles.stats()

,total,empty,distinct,uniqueness,entropy
ARREST_KEY,115299,0,115299,1.000000,16.815020
ARREST_DATE,115299,0,273,0.002368,8.047698
PD_CD,115299,21,252,0.002186,5.401154
PD_DESC,115299,61,233,0.002022,5.366524
KY_CD,115299,61,66,0.000573,4.534242
OFNS_DESC,115299,61,63,0.000547,4.339051
LAW_CODE,115299,0,922,0.007997,6.328932
LAW_CAT_CD,115299,1078,4,0.000035,1.032090
ARREST_BORO,115299,0,5,0.000043,2.172560
ARREST_PRECINCT,115299,0,77,0.000668,6.119069


In [70]:
profiles.minmax('ARREST_KEY')

,min,max
int,222471128,234352983


In [71]:
ds_full.distinct('PD_CD')

Counter({'105': 2106,
         '157': 258,
         '109': 8591,
         '101': 14678,
         '582': 1,
         '397': 5678,
         '244': 3208,
         '166': 9,
         '153': 149,
         '263': 81,
         '748': 1815,
         '515': 937,
         '779': 5256,
         '681': 1351,
         '729': 2444,
         '293': 103,
         '155': 52,
         '177': 258,
         '339': 9110,
         '175': 716,
         '750': 879,
         '490': 534,
         '922': 3728,
         '511': 3197,
         '904': 275,
         '847': 558,
         '259': 2540,
         '113': 3296,
         '792': 3461,
         '705': 2956,
         '439': 4100,
         '782': 1320,
         '493': 150,
         '969': 1109,
         '509': 31,
         '639': 2165,
         '268': 2617,
         '129': 35,
         '802': 279,
         '117': 580,
         '548': 66,
         '503': 1852,
         '649': 67,
         '49': 974,
         '209': 292,
         '462': 350,
         '409': 56,
  

In [72]:
profiles.minmax('PD_CD')

,min,max
int,0,997


### Validating PD_CD Column by removing empty values

In [77]:
def validate_PD(data,column_name):
    data = data[data[column_name].between(1,998)]
    return data

    
##ds_full = ds_full.update('PD_CD', validate_PD_CD)

In [78]:
## ds_full = ds_full.update('CRM_ATPT_CPTD_CD', validate_crime_indicator)
column12 = validate_PD(ds_full,'PD_CD')
#column8[column8['VIC_AGE_GROUP'] == ""]
column12

TypeError: 'DataPipeline' object is not subscriptable